# Install openai-agents SDK

In [2]:
!pip install -Uq openai-agents  "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


# Make your Jupyter Notebook capable of running asynchronous functions.

In [3]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with LiteLLm and OPENAI-Agent SDK

## Function Run Sync

In [4]:


import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")






def main(model: str, api_key: str):
  agent = Agent(
      name="Assistant",
      instructions="You only respond in AI.",
      model=LitellmModel(model=MODEL, api_key=api_key),

  )

  result = Runner.run_sync(agent, "weather in karachi")
  print(result.final_output)


main(model=MODEL, api_key=GEMINI_API_KEY)



The weather in Karachi is currently 32°C and mostly clear. The wind is blowing from the southwest at 18 km/h.


# Async Function

# tool celling

In [9]:
import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")



@function_tool
def get_weather(city: str)->str:
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."


async def main(model: str, api_key: str):
  agent = Agent(
      name="Assistant",
      instructions="You only respond in haikus.",
      model=LitellmModel(model=model, api_key=api_key),
      tools=[get_weather]

  )

  result = await Runner.run(agent, "What's the weather in karachi?")
  print(result.final_output)


asyncio.run(main(model=MODEL, api_key=GEMINI_API_KEY))



[debug] getting weather for karachi
The sun shines brightly,
Karachi feels the warm rays,
A sunny weather.



# ***handoofs***


In [29]:
from ast import Await
import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")



@function_tool
def get_weather(city: str)->str:
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."
@function_tool
def get_ai():
  return "this is ai"


weather_agent = Agent(
  name="WeatherAssistant",
  instructions="You only respond in weather.",
  model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
  handoff_description="weather assistant",
  tools=[get_weather]

)
ai_agent = Agent(
  name="AiAssistant",
  instructions="You only respond in AI information.",
  model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
  handoff_description="ai assistant",
  tools=[get_ai]
)

agent = Agent(
  name="Assistant",
  instructions="You only respond in haikus.",
  model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
  handoffs=[weather_agent,ai_agent]


)




result = await Runner.run(agent,"what is a ai")
print(result.final_output)

I can't define that.
I can get ai help, though,
If you would like that.



In [30]:
result.last_agent.name

'Assistant'

In [34]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

# Disable tracing
set_tracing_disabled(disabled=True)

# Model and API Key
MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

# Tool: Weather
@function_tool
def get_weather(city: str) -> str:
    print(f"[debug] Getting weather for {city}")
    return f"The weather in {city} is sunny."

# Tool: AI Info
@function_tool
def get_ai() -> str:
    return "This is AI, the simulation of human intelligence in machines."

# Sub-agent: WeatherAssistant
weather_agent = Agent(
    name="WeatherAssistant",
    instructions="You only respond to weather-related questions. Use the get_weather tool.",
    model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoff_description="Answers weather-related questions",
    tools=[get_weather]
)

# Sub-agent: AiAssistant
ai_agent = Agent(
    name="AiAssistant",
    instructions="You only respond to AI-related questions. Use the get_ai tool.",
    model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoff_description="Answers AI-related questions",
    tools=[get_ai]
)

# Main Agent: Decides whether to handoff to weather_agent or ai_agent
agent = Agent(
    name="SmartAssistant",
    instructions="If the question is about weather, hand it off to WeatherAssistant. If it's about AI, hand it off to AiAssistant.",
    model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoffs=[weather_agent, ai_agent]
)

result = await Runner.run(agent,"what is the weather is karachi")
print(result.final_output)
print(result.last_agent.name)


result = await Runner.run(agent,"what is a ai")
print(result.final_output)
print(result.last_agent.name)

result = await Runner.run(agent,"hi")
print(result.final_output)
print(result.last_agent.name)


[debug] Getting weather for karachi
The weather in karachi is sunny.

WeatherAssistant
I am an AI assistant. How can I help you today?

AiAssistant
How can I help you today?

SmartAssistant


In [42]:
from ast import Await  # This line is incorrect and not needed
import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

# Disable tracing for performance or debugging
set_tracing_disabled(disabled=True)

# Model & API Key
MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

# ----------- Tools -----------

@function_tool
def get_weather(city: str) -> str:
    print(f"[debug] Getting weather for {city}")
    return f"The weather in {city} is currently sunny with a mild breeze."

@function_tool
def get_ai() -> str:
    return "AI (Artificial Intelligence) refers to the simulation of human intelligence by machines."

# ----------- Sub Agents -----------

weather_agent = Agent(
    name="WeatherAssistant",
    instructions=(
        "You are a weather assistant. "
        "Only provide accurate and concise weather-related responses. "
        "Do not answer anything unrelated to weather."
    ),
    model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoff_description="Handles weather-related queries",
    tools=[get_weather]
)

ai_agent = Agent(
    name="AiAssistant",
    instructions=(
        "You are an AI information expert. "
        "Only respond to questions about Artificial Intelligence in a clear and informative way."
    ),
    model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoff_description="Handles AI-related queries",
    tools=[get_ai]
)

# ----------- Main Agent -----------

main_agent = Agent(
    name="Assistant",
    instructions=(
        "You are a poetic assistant who speaks in haikus. "
        "If the user query is about weather or AI, hand it off to the appropriate specialized agent."
    ),
    model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
    handoffs=[weather_agent, ai_agent]
)

# ----------- Run the Agent -----------

result = await Runner.run(agent,"what is the weather is karachi")
print(result.final_output)
print(result.last_agent.name)


result = await Runner.run(agent,"ai kia ha or isa kasa sikha")
print(result.final_output)
print(result.last_agent.name)

result = await Runner.run(agent,"hi")
print(result.final_output)
print(result.last_agent.name)

To weather assistant,
I must send your simple quest.
Awaiting results.

Assistant
To AI assistant,
I must pass your query now,
For its deep wisdom.

Assistant
A greeting to you,
Words soft as the falling leaves,
May peace fill your day.

Assistant
